In [1]:
# preliminaries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd

In [2]:
# Run this command to print files in directory; useful for knowing exact names of files for importing
import os
print(os.getcwd())
print("\n", os.listdir()) # list items in current working directory after "\n" return key

C:\Users\mgrou\Documents\GitHub\final_project\ciliary_proteomes_and_js_exomes

 ['.git', '.ipynb_checkpoints', 'Cellbody_and_cilia+_fraction.xlsx', 'cellBody_only.xlsx', 'CellBody_only_Repaired.xlsx', 'Cheng_Grout_final_project.ipynb', 'Cilia_only_fraction.xlsx', 'Experimental_Copy.ipynb', 'JSexomes_1_1percentExAC_reAnn138_output_new.txt', 'JSexomes_2_output_1percentExAC.xlsx', 'js_genes.txt', 'README.md']


In [3]:
# import 2nd round of JS exomes
exome_2 = pd.read_excel("JSexomes_2_output_1percentExAC.xlsx")

In [4]:
# import 1st round of JS exome data
exome_1 = pd.read_table("JSexomes_1_1percentExAC_reAnn138_output_new.txt")

In [5]:
# Merge old and new exome data sets together into one pd df
exome = pd.concat([exome_1, exome_2])
exome.columns
print(exome.shape)
print(exome_1.shape)
print(exome_2.shape)

(449531, 46)
(293408, 40)
(156123, 39)


In [6]:
# Print column names of new exomes df
exome.columns

Index(['AB', 'AC_Hmz(,Hemi)', 'AC_Home', 'AN_Adj', 'Alt', 'AlternateAllele',
       'AminoAcids', 'CADD_PHRED_score', 'CADD_score', 'CDNA_Position',
       'Chromosome', 'Dist2Splice', 'ESP_AA_Frequency', 'ESP_Afr_Frequency',
       'ESP_All_Frequency', 'ESP_Asn_Frequency', 'ESP_EA_Frequency',
       'ESP_Eur_Frequency', 'ExAC_All_MAF', 'ExAC_Ethnicity', 'ExAC_Filter',
       'ExAC_Highest_MAF', 'ExAC_NFE_MAF', 'File Name', 'Filter', 'GERP',
       'GVSfunction', 'Gene', 'Genotype', 'P1Alt', 'P1Ref', 'P1Total',
       'P1_AltFreq', 'P2Alt', 'P2Ref', 'P2Total', 'P2_AltFreq', 'Polyphen',
       'Position', 'Proband_Sample_Name', 'Protein_Position', 'Rarity_type',
       'Ref', 'ReferenceAllele', 'Total_RD', 'rsID'],
      dtype='object')

In [22]:
# From all exome data, pull out only columns we want.

# 'Gene' for us to later match up a variant to a data point by gene
# If we are matching up data points back to exome variants exactly, we should use all chr, pos, ref, and alt
# If we are simiply matching a data point back up to the gene, not necessarily WHICH VARIANT in exome data it is, then we don't need chr
# Dist2Splice, CADD_score, and and ExAC value are all great. We want ExAC_Highest_MAF because this is going to be the demographic
# category for which the allele frequency was the highest, out of all measured demographics. Often this is EXNF = 0 for ALL groups,but
# not always.
# Do we want to include AB (allele balance?) What information could this tell us in the assays we'll be doing? (Just something
# for us to think about. I don't think it will hurt to have it in there.)

# I changed the code a bit because Pandas was yelling at us for using direct (?) indexing over .loc[] instead.
# We know from experience that the only place that 'EXNF' should appear in any column is in the ExAC-related ones,
# so it should be no problem to replace that string w/ 0 in the whole df.

exomeFiltered = exome.loc[:, ['Gene', 'CADD_score', 'AB', 'Chromosome', 'Dist2Splice', 'ExAC_Highest_MAF']].replace('EXNF',0)

In [23]:
print(exomeFiltered.head())

      Gene CADD_score    AB Chromosome  Dist2Splice ExAC_Highest_MAF
0    DNMBP      0.647  1.00         10         66.0                0
1  CWF19L1      11.65  0.45         10         80.0                0
2  C10orf2      3.704  0.45         10          3.0                0
3    LZTS2      10.92  0.60         10        144.0       0.00009806
4    PDZD7      2.215  0.44         10         14.0       0.00017013


In [45]:
CellBodyOnly = pd.read_excel("CellBody_only_Repaired.xlsx")
CiliaOnly = pd.read_excel("Cilia_only_fraction.xlsx")
BodyAndCilia = pd.read_excel("Cellbody_and_cilia+_fraction.xlsx")
CellBodyOnlyGenes = CellBodyOnly['Gene names']
CiliaOnlyGenes = CiliaOnly['Gene names']
BodyAndCiliaGenes = BodyAndCilia['Gene names']

In [121]:
Counter = 0
Genes = exomeFiltered['Gene']
subsetGene = Genes[:3000]
index = np.zeros(len(subsetGene))
columns = ['Localization']
LocalizationPosition = pd.DataFrame(index=index,columns=columns)
#LocalizationPosition = LocalizationPosition.fillna(0)

In [122]:
LocalizationPositionArray = np.zeros(len(subsetGene))

In [114]:
for x in range(len(subsetGene)):
    if CellBodyOnlyGenes.isin(subsetGene[x].split())[0]:
        LocalizationPosition[x] = 1
    elif CiliaOnlyGenes.isin(subsetGene[x].split())[0]:
        LocalizationPosition[x] = 2
    elif BodyAndCiliaGenes.isin(subsetGene[x].split())[0]:
        LocalizationPosition[x] = 3
    else:
        LocalizationPosition[x] = 0
print("done")

done


In [123]:
for x in range(len(subsetGene)):
    if CellBodyOnlyGenes.isin(subsetGene[x].split())[0]:
        LocalizationPositionArray[x] = 1
    elif CiliaOnlyGenes.isin(subsetGene[x].split())[0]:
        LocalizationPositionArray[x] = 2
    elif BodyAndCiliaGenes.isin(subsetGene[x].split())[0]:
        LocalizationPositionArray[x] = 3
    else:
        LocalizationPositionArray[x] = 4
print("done")
LocalizationPositionArray

done


array([ 4.,  4.,  4., ...,  4.,  4.,  4.])

In [ ]:
exomeFiltered['Localization_Position'] = LocalizationPosition #run after finished with loading LocalizationPosition

In [76]:
for x in range(len(Genes)):
    if CellBodyOnlyGenes.isin(Genes[x])[0]:
        LocalizationPosition[Counter] = 1
    elif CiliaOnlyGenes.isin(Genes[x])[0]:
        LocalizationPosition[Counter] = 2
    elif BodyAndCiliaGenes.isin(Genes[x])[0]:
        LocalizationPosition[x] = 3
    else:
        LocalizationPosition[x] = 0
print("done")

MemoryError: 

In [ ]:
for x in range(len(Genes)):
    if CellBodyOnlyGenes.isin(Genes[Counter])[0]:
        LocalizationPosition[Counter] = 0
   # elif Genes[Counter] == CiliaOnlyGenes:
   #     LocalizationPosition[Counter] = 1
   # elif Genes[Counter] == BodyAndCiliaGenes:
   #     LocalizationPosition[Counter] = 2
   # else:
   #     LocalizationPosition[Counter] = NaN
    Counter += 1
print("done")